In [1]:
import geopandas as gpd
from shapely.geometry import box
from osgeo import gdal
from copy import deepcopy
import os, math
import numpy as np
import requests
import pandas as pd
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import zipfile
import warnings

In [2]:
os.chdir(r"C:\Users\brenp\Downloads")

In [3]:
# Definir ruta y descargar el archivo ZIP
url = "https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.88/win64/chromedriver-win64.zip"
zip_path = "chromedriver-win64.zip"
extract_path = "chromedriver-win64"

response = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(response.content)
    
if zipfile.is_zipfile(zip_path):
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)
    os.remove(zip_path) 

In [4]:
# Ruta al WebDriver extraído
webdriver_path = r'chromedriver-win64\chromedriver-win64\chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('headless=False')
options.add_argument('--disable-gpu')
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service, options=options)

In [5]:
LIMITE_MUNICIPAL = gpd.read_file("https://raw.githubusercontent.com/andimcfl/DESGARGAS_INEGI/main/scrips/INSUMOS%20BASE/00mun.geojson")
print("Carga completa")

def buscar_municipio(nom_ent, nomgeo):
    resultado = LIMITE_MUNICIPAL[(LIMITE_MUNICIPAL['NOM_ENT'] == nom_ent) & (LIMITE_MUNICIPAL['NOMGEO'] == nomgeo)]    
    if not resultado.empty:
        resultado = resultado[['CVE_ENT', 'NOM_ENT', 'NOMGEO', 'CVEGEO']]
        print(f"Municipio encontrado:\n{resultado}")
    else:
        print(f"No se encontró el municipio con ESTADO '{nom_ent}' y MUNICIPIO '{nomgeo}'.")
    
    return resultado

Carga completa


In [6]:
nom_ent = 'Ciudad de México'
nomgeo = 'Tlalpan'
resultado = buscar_municipio(nom_ent, nomgeo)

Municipio encontrado:
    CVE_ENT           NOM_ENT   NOMGEO CVEGEO
282      09  Ciudad de México  Tlalpan  09012


In [7]:
cvegeo = '09012'
cve_ent = '09'
municipio = LIMITE_MUNICIPAL[LIMITE_MUNICIPAL['CVEGEO'] == cvegeo].to_crs(epsg=6372)
bounds = municipio.total_bounds
bounding_box = box(bounds[0], bounds[1], bounds[2], bounds[3])
buffer = bounding_box.buffer(2000) #área del buffer par recortar
#zona_corte = gpd.GeoDataFrame(geometry=[buffer], crs=LIMITE_MUNICIPAL.crs).to_crs(epsg=6372) 
#zona_corte = gpd.GeoDataFrame(geometry=[bounding_box], crs=LIMITE_MUNICIPAL.crs).to_crs(epsg=6372) 
zona_corte = municipio
print(zona_corte)

    CVEGEO CVE_ENT CVE_MUN   NOMGEO           NOM_ENT  \
282  09012      09     012  Tlalpan  Ciudad de México   

                                              geometry  
282  MULTIPOLYGON (((2791349.819 815583.059, 279134...  


In [8]:
zona_corte.to_file(f"C{cvegeo}_ZONACORTE_EP_2025.geojson")

In [9]:
cartas = gpd.read_file("https://raw.githubusercontent.com/andimcfl/DESGARGAS_INEGI/main/scrips/INSUMOS%20BASE/div50k_lcc.geojson").to_crs(epsg=6372)
print("Carga completa")
claves = cartas['CLAVE50K'].unique()
claves_en_zona_corte = gpd.sjoin(cartas, zona_corte, how='inner', predicate='intersects')
claves_cartas_topograficas = claves_en_zona_corte['CLAVE50K'].unique()
print(claves_cartas_topograficas)

Carga completa
['E14A39' 'E14A49']


In [10]:
url = 'https://www.inegi.org.mx/programas/topografia/50000/#descargas'
driver.get(url)

In [12]:
wait = WebDriverWait(driver, 100)

wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input#bcm_resultBusqueda3_input_search')))

clave_enlace_list = []

def buscar_clave(clave):
    search_box = driver.find_element(By.CSS_SELECTOR, 'input#bcm_resultBusqueda3_input_search')
    search_button = driver.find_element(By.CSS_SELECTOR, 'button#bcm_buscar_btn_resultBusqueda3')
    driver.execute_script("arguments[0].scrollIntoView();", search_box)
    driver.execute_script("arguments[0].scrollIntoView();", search_button)
    search_box.clear()
    search_box.send_keys(clave)
    WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#bcm_buscar_btn_resultBusqueda3')))
    driver.execute_script("arguments[0].click();", search_button)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody')))

def obtener_Serie (row):
    Serie = None
    for i in [4]:
        try:
            Serie_element = row.find_element(By.CSS_SELECTOR, f'td:nth-child({i}) .tablesaw-cell-content')
            Serie = Serie_element.text.strip()
            if Serie: 
                break
        except Exception:
            continue 
    return Serie if Serie else "Sin Serie"

def procesar_filas(clave_buscar):
    rows = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')
    for row in rows:
        try:
            clave_element = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1) .tablesaw-cell-content')
            clave = clave_element.text.strip()
            Serie = obtener_Serie(row) 
            enlace_descarga = None
            for i in [8, 9, 10]:
                try:
                    tipo_archivo_element = row.find_element(By.CSS_SELECTOR, f'td:nth-child({i}) img[src="/img/ico/ico_shp.png"]')
                    if tipo_archivo_element:
                        enlace_element = row.find_element(By.CSS_SELECTOR, f'td:nth-child({i}) a[href$=".zip"]')
                        enlace_descarga = enlace_element.get_attribute('href')
                        break
                except Exception:
                    continue
            if enlace_descarga:
                clave_enlace_list.append((clave, Serie, enlace_descarga))
        except Exception:
            continue

for clave_buscar in claves_cartas_topograficas:
    buscar_clave(clave_buscar)
    time.sleep(15)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        procesar_filas(clave_buscar)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(20)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

if clave_enlace_list:
    for clave, Serie, enlace in clave_enlace_list:
        print(f"Clave: {clave} - Serie: {Serie} - Enlace de descarga: {enlace}")
else:
    print("No se encontraron resultados para las claves buscadas")

Clave: E14A39 - Serie: Sin Serie - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_50_000/889463854135_s.zip
Clave: E14A39 - Serie: 2019 - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_50_000/889463833406_s.zip
Clave: E14A39 - Serie: III - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_50_000/702825266721_s.zip
Clave: E14A39 - Serie: II - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_50_000/702825003701_s.zip
Clave: E14A49 - Serie: Sin Serie - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_50_000/8894638

In [13]:
driver.quit()

In [16]:
def descargar_archivo(url, carpeta_destino, nombre_archivo):
    respuesta = requests.get(url, stream=True)
    ruta_archivo = os.path.join(carpeta_destino, nombre_archivo)
    
    with open(ruta_archivo, 'wb') as archivo:
        for chunk in respuesta.iter_content(chunk_size=32768):
            if chunk:
                archivo.write(chunk)
    
    print(f"Descargado: {ruta_archivo}")

Serie_deseada = "III"
carpeta_descargas = "CARTAS TOPOGRAFICAS"
os.makedirs(carpeta_descargas, exist_ok=True)

for clave, Serie, enlace in clave_enlace_list:
    if Serie == Serie_deseada:
        nombre_archivo = f"{clave}_{Serie}.zip"
        descargar_archivo(enlace, carpeta_descargas, nombre_archivo)

print("Descarga completa.")

Descargado: CARTAS TOPOGRAFICAS\E14A39_III.zip
Descargado: CARTAS TOPOGRAFICAS\E14A49_III.zip
Descarga completa.


In [17]:
carpeta_zip = 'CARTAS TOPOGRAFICAS'
os.makedirs(carpeta_zip, exist_ok=True)

for archivo_zip in os.listdir(carpeta_zip):
    if archivo_zip.endswith('.zip'):
        ruta_zip = os.path.join(carpeta_zip, archivo_zip)
        nombre = os.path.splitext(archivo_zip)[0]
        carpeta = os.path.join(carpeta_zip, nombre)
        os.makedirs(carpeta, exist_ok=True)
        with zipfile.ZipFile(ruta_zip, 'r') as archivo_zip_obj:
            archivo_zip_obj.extractall(carpeta)
            print(f"Descomprimido: {ruta_zip} en {carpeta}")

print("Descompresión completa.")

Descomprimido: CARTAS TOPOGRAFICAS\E14A39_III.zip en CARTAS TOPOGRAFICAS\E14A39_III
Descomprimido: CARTAS TOPOGRAFICAS\E14A49_III.zip en CARTAS TOPOGRAFICAS\E14A49_III
Descompresión completa.


In [18]:
def buscar_archivos_shp(carpeta):
    archivos_shp = []    
    for raiz, directorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith('.shp'):
                ruta_archivo = os.path.join(raiz, archivo)
                archivos_shp.append(ruta_archivo)    
    return archivos_shp
    
for carpeta in os.listdir(carpeta_zip):
    ruta_carpeta = os.path.join(carpeta_zip, carpeta)    
    if os.path.isdir(ruta_carpeta):
        ruta_conjunto_datos = os.path.join(ruta_carpeta, 'conjunto_de_datos')        
        if os.path.isdir(ruta_conjunto_datos):
            archivos_shp = buscar_archivos_shp(ruta_conjunto_datos)            
            if archivos_shp:
                print(f"Archivos .shp encontrados en {ruta_conjunto_datos}:")
                for archivo in archivos_shp:
                    print(f"  - {archivo}")
            else:
                print(f"No se encontraron archivos .shp en {ruta_conjunto_datos}.")

Archivos .shp encontrados en CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos:
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\acueducto50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\aeropuerto50_a.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\area_cultiv50_a.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\area_ver_ur50_a.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\ban_mat50_p.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\bordo50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\calle50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\camino50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\canal50_a.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\canal50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\carretera50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\caseta_peaj50_p.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\cementerio50_a.shp
  - CARTAS 

In [19]:
nombres_archivos = ['acueducto50_l.shp', 'canal50_l.shp', 'corriente_ag50_l.shp','cuerpo_agua50_a.shp', 'curva_nivel50_l.shp', 'linea_trans50_l.shp','canal50_a.shp', 'conducto50_l.shp', 'via_ferrea50_l.shp','aeropuerto50_a.shp']

def buscar_archivos(carpeta, nombres_archivos):
    archivos_encontrados = {nombre: [] for nombre in nombres_archivos}
    for raiz, directorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo in nombres_archivos:
                archivos_encontrados[archivo].append(os.path.join(raiz, archivo))
    return archivos_encontrados
    
carpeta_zip = 'CARTAS TOPOGRAFICAS'

gdfs_dict = {nombre: [] for nombre in nombres_archivos}

for carpeta in os.listdir(carpeta_zip):
    ruta = os.path.join(carpeta_zip, carpeta)
    if os.path.isdir(ruta):
        ruta_conjunto_de_datos = os.path.join(ruta, 'conjunto_de_datos')
        if os.path.isdir(ruta_conjunto_de_datos):
            archivos_encontrados = buscar_archivos(ruta_conjunto_de_datos, nombres_archivos)            
            for nombre_archivo, rutas_archivos in archivos_encontrados.items():
                for ruta_archivo in rutas_archivos:
                    gdf = gpd.read_file(ruta_archivo)
                    gdf = gdf.to_crs(epsg=6372)
                    gdf['Cve_Ent'] = cve_ent
                    gdf['Cve_MunC'] = cvegeo
                    campos_a_conservar = [campo for campo in ['geografico', 'elevacion', 'nombre'] if campo in gdf.columns]
                    gdf = gdf[['geometry', 'Cve_Ent', 'Cve_MunC'] + campos_a_conservar]
                    gdfs_dict[nombre_archivo].append(gdf)
                    
for nombre_archivo, gdfs in gdfs_dict.items():
    if gdfs:
        gdf_unido = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
        if gdf_unido.crs != 'EPSG:6372':
            gdf_unido = gdf_unido.to_crs(epsg=6372)        
        gdf_cortado = gpd.overlay(gdf_unido, zona_corte, how='intersection')        
        if not gdf_cortado.empty:
            nombre_archivo_mayus = nombre_archivo.upper()
            nombre_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_mayus)[0]}_INEGI_Serie_III.geojson"
            ruta_salida = os.path.join(carpeta_zip, nombre_salida)
            gdf_cortado.to_file(ruta_salida)
            print(f"Archivo unido y cortado guardado en: {ruta_salida}")
        else:
            print(f"El GeoDataFrame resultante de {nombre_archivo} está vacío después del corte.")
print("Proceso de unión y corte completo.")

Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09012_ACUEDUCTO50_L_INEGI_Serie_III.geojson
El GeoDataFrame resultante de canal50_l.shp está vacío después del corte.
Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09012_CORRIENTE_AG50_L_INEGI_Serie_III.geojson
Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09012_CUERPO_AGUA50_A_INEGI_Serie_III.geojson
Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09012_CURVA_NIVEL50_L_INEGI_Serie_III.geojson
Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09012_LINEA_TRANS50_L_INEGI_Serie_III.geojson
El GeoDataFrame resultante de canal50_a.shp está vacío después del corte.
Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09012_CONDUCTO50_L_INEGI_Serie_III.geojson
Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09012_VIA_FERREA50_L_INEGI_Serie_III.geojson
El GeoDataFrame resultante de aeropuerto50_a.shp está vacío después del corte.
Proceso de unión y corte completo.
